In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bbbp_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bbbp/chemberta2_valid2_bbbp_3_predictions.csv')
bbbp_molformer_valid2 = pd.read_csv('./molformer/results/bbbp/molformer_valid2_bbbp_3_epoch29.csv')
bbbp_molbert_valid2 = pd.read_csv('./molbert/results/bbbp/molbert_valid2_bbbp_3.csv')

# Load the test data for each model
bbbp_chemberta2_test = pd.read_csv('./chemberta2/results/bbbp/chemberta2_test_bbbp_3_predictions.csv')
bbbp_molformer_test = pd.read_csv('./molformer/results/bbbp/molformer_test_bbbp_3_epoch29.csv')
bbbp_molbert_test = pd.read_csv('./molbert/results/bbbp/molbert_test_bbbp_3.csv')

# features

# Load the features from chemberta
bbbp_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bbbp/chemberta2_valid2_bbbp_3_features.csv')
bbbp_chemberta2_features_test = pd.read_csv('./chemberta2/features/bbbp/chemberta2_test_bbbp_3_features.csv')

# Load the features from molformer
bbbp_molformer_features_valid2 = pd.read_csv('./molformer/features/bbbp/molformer_valid2_bbbp_3_features.csv')
bbbp_molformer_features_test = pd.read_csv('./molformer/features/bbbp/molformer_test_bbbp_3_features.csv')

# Load the features from molbert
bbbp_molbert_features_valid2 = pd.read_csv('./molbert/features/bbbp/molbert_valid2_bbbp_3_features.csv')
bbbp_molbert_features_test = pd.read_csv('./molbert/features/bbbp/molbert_test_bbbp_3_features.csv')

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bbbp_chemberta_actual = bbbp_chemberta2_test['p_np']
bbbp_chemberta_pred = bbbp_chemberta2_test['y_pred']
bbbp_chemberta_probs = bbbp_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bbbp_molformer_actual = bbbp_molformer_test['Actual']
bbbp_molformer_pred = (bbbp_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bbbp_molformer_probs = bbbp_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bbbp_molbert_actual = bbbp_molbert_test['target']
bbbp_molbert_pred = bbbp_molbert_test['pred']
bbbp_molbert_probs = bbbp_molbert_test['prob']

# Calculating metrics
bbbp_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bbbp_chemberta_actual, bbbp_chemberta_pred, bbbp_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bbbp_molformer_actual, bbbp_molformer_pred, bbbp_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bbbp_molbert_actual, bbbp_molbert_pred, bbbp_molbert_probs)]:
    bbbp_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bbbp_metrics_results

{'Chemberta2': {'Accuracy': 0.8292682926829268,
  'F1 Score': 0.8844884488448845,
  'ROC-AUC': 0.9488724127278345,
  'PR-AUC': 0.9882303014277589},
 'Molformer': {'Accuracy': 0.8774509803921569,
  'F1 Score': 0.924924924924925,
  'ROC-AUC': 0.8789632213062777,
  'PR-AUC': 0.965741487133955},
 'Molbert': {'Accuracy': 0.9068627450980392,
  'F1 Score': 0.9425981873111783,
  'ROC-AUC': 0.9514901712111604,
  'PR-AUC': 0.9885586344415213}}

In [3]:
# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molformer_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molformer_valid2['smiles'])
print(f"Missing smiles in molformer_valid2: {missing_smiles_molformer_valid2}")

# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molbert_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molbert_valid2['smiles'])
print(f"Missing smiles in molbert_valid2: {missing_smiles_molbert_valid2}")

# Combine the invalid indices from molformer_valid2 with the missing indices from molbert_valid2
combined_invalid_smiles_valid2 = list(set(missing_smiles_molformer_valid2).union(set(missing_smiles_molbert_valid2)))

print(f"These indices will be removed from the valid2 set: {combined_invalid_smiles_valid2}")

Missing smiles in molformer_valid2: {'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 's1cc(nc1\\[NH]=C(\\N)N)C', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br'}
Missing smiles in molbert_valid2: {'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 's1cc(nc1\\[NH]=C(\\N)N)C', '[Na+].Cc1sc(SCC2=C(N3[C@H](SC2)[C@H](NC(=O)Cn4cnnn4)C3=O)C([O-])=O)nn1', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', '[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](NC(=O)C3=CC=C(NC3=O)c4ccc(cc4)[S](=O)(=O)N(CCO)CCO)c5ccc(O)cc5)C(=O)N2[C@H]1C([O-])=O', '[Na+].Cn1nnnc1SCC2=C(N3[C@H](SC2)[C@H](NC(=O)CSC(F)(F)F)C3=O)C([O-])=O', '[Na+].CO\\N=C(C(=O)N[C@@H]1[C@@H]2SCC(=C(N2C1=O)C([O-])=O)COC(C)=O)\\c3csc(N)n3'}
These indices will be removed from the valid2 set: ['c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)

In [4]:
# Function to remove invalid SMILES
def remove_invalid_smiles(df, invalid_smiles_list):
    return df[~df['smiles'].isin(invalid_smiles_list)]

# Remove invalid SMILES from each dataframe
bbbp_chemberta2_valid2 = remove_invalid_smiles(bbbp_chemberta2_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_valid2 = remove_invalid_smiles(bbbp_molformer_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_valid2 = remove_invalid_smiles(bbbp_molbert_valid2, combined_invalid_smiles_valid2)
bbbp_chemberta2_features_valid2 = remove_invalid_smiles(bbbp_chemberta2_features_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_features_valid2 = remove_invalid_smiles(bbbp_molformer_features_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_features_valid2 = remove_invalid_smiles(bbbp_molbert_features_valid2, combined_invalid_smiles_valid2)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_valid2.shape)
print(bbbp_molformer_valid2.shape)
print(bbbp_molbert_valid2.shape)
print(bbbp_chemberta2_features_valid2.shape)
print(bbbp_molformer_features_valid2.shape)
print(bbbp_molbert_features_valid2.shape)

(401, 8)
(401, 6)
(401, 4)
(401, 386)
(401, 769)
(401, 769)


In [5]:
bbbp_y_ensemble_valid2 = bbbp_chemberta2_valid2['p_np']

# Convert the ensemble target to a Series if not already done
bbbp_y_ensemble_valid2_s = pd.Series(bbbp_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_valid2['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)

bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_valid2['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)

bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_valid2['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features bbbp_chemberta2_features_valid2.iloc[:, 2:]
bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_valid2.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)

bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_valid2.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)

bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_valid2.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)

# bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

# Function to calculate BCE for each row with epsilon to avoid log(0)
def calculate_bce_rowwise(y_true, y_pred, epsilon=1e-12):
    # Clip y_pred to ensure it's between epsilon and 1-epsilon
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
bbbp_X_ensemble_valid2 = pd.concat([bce_loss_df, train_bbbp_prob], axis=1)

In [6]:
import numpy as np
import pandas as pd
from skglm import GroupLasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([bbbp_chemberta2_prob, bbbp_chemberta2_features], axis=1)
molformer_X = pd.concat([bbbp_molformer_prob, bbbp_molformer_features], axis=1)
molbert_X = pd.concat([bbbp_molbert_prob, bbbp_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

# Define groups for each dataset
n_features_chemberta = chemberta_X_scaled.shape[1]
n_features_molformer = molformer_X_scaled.shape[1]
n_features_molbert = molbert_X_scaled.shape[1]

# Example grouping structure: 1st feature is the probability, rest are features
chemberta_groups = [list(range(0, 1)), list(range(1, n_features_chemberta))]
molformer_groups = [list(range(0, 1)), list(range(1, n_features_molformer))]
molbert_groups = [list(range(0, 1)), list(range(1, n_features_molbert))]

# Initialize the GroupLasso models
group_lasso_chemberta = GroupLasso(groups=chemberta_groups, alpha=1.0, tol=0.0001)
group_lasso_molformer = GroupLasso(groups=molformer_groups, alpha=1.0, tol=0.0001)
group_lasso_molbert = GroupLasso(groups=molbert_groups, alpha=1.0, tol=0.0001)

# Setup cross-validation for alpha tuning using neg_log_loss for BCE
param_grid = {'alpha': np.logspace(-4, 1, 10)}

cv_chemberta = GridSearchCV(estimator=group_lasso_chemberta, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molformer = GridSearchCV(estimator=group_lasso_molformer, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molbert = GridSearchCV(estimator=group_lasso_molbert, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit GridSearchCV for each dataset with BCE as the target
cv_chemberta.fit(chemberta_X_scaled, chemberta_y_bce)
cv_molformer.fit(molformer_X_scaled, molformer_y_bce)
cv_molbert.fit(molbert_X_scaled, molbert_y_bce)

# Retrieve the best models and parameters
best_model_chemberta = cv_chemberta.best_estimator_
best_model_molformer = cv_molformer.best_estimator_
best_model_molbert = cv_molbert.best_estimator_

In [7]:
# identify missing in test
missing_smiles_molformer_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molformer_test['smiles'])
print(f"Missing smiles in molformer_test: {missing_smiles_molformer_test}")

missing_smiles_molbert_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molbert_test['smiles'])
print(f"Missing smiles in molbert_test: {missing_smiles_molbert_test}")

# Combine the invalid smiles from molformer_test with the missing smiles from molbert_test
combined_invalid_smiles_test = list(set(missing_smiles_molformer_test).union(set(missing_smiles_molbert_test)))
print(f"These indices will be removed from the test set: {combined_invalid_smiles_test}")

Missing smiles in molformer_test: {'n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O'}
Missing smiles in molbert_test: {'n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O'}
These indices will be removed from the test set: ['n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O']


In [8]:
# Remove invalid SMILES from each dataframe
bbbp_chemberta2_test = remove_invalid_smiles(bbbp_chemberta2_test, combined_invalid_smiles_test)
bbbp_molformer_test = remove_invalid_smiles(bbbp_molformer_test, combined_invalid_smiles_test)
bbbp_molbert_test = remove_invalid_smiles(bbbp_molbert_test, combined_invalid_smiles_test)
bbbp_chemberta2_features_test = remove_invalid_smiles(bbbp_chemberta2_features_test, combined_invalid_smiles_test)
bbbp_molformer_features_test = remove_invalid_smiles(bbbp_molformer_features_test, combined_invalid_smiles_test)
bbbp_molbert_features_test = remove_invalid_smiles(bbbp_molbert_features_test, combined_invalid_smiles_test)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_test.shape)
print(bbbp_molformer_test.shape)
print(bbbp_molbert_test.shape)
print(bbbp_chemberta2_features_test.shape)
print(bbbp_molformer_features_test.shape)
print(bbbp_molbert_features_test.shape)

(204, 8)
(204, 6)
(204, 4)
(204, 386)
(204, 769)
(204, 769)


In [9]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
bbbp_chemberta2_prob_test = pd.DataFrame({'chemberta2': bbbp_chemberta2_test['softmax_class_1_prob']})
bbbp_chemberta2_prob_test.reset_index(drop=True, inplace=True)

bbbp_molformer_prob_test = pd.DataFrame({'molformer': bbbp_molformer_test['Prob_Class_1']})
bbbp_molformer_prob_test.reset_index(drop=True, inplace=True)

bbbp_molbert_prob_test = pd.DataFrame({'molbert': bbbp_molbert_test['prob']})
bbbp_molbert_prob_test.reset_index(drop=True, inplace=True)

bbbp_chemberta2_features_t = pd.DataFrame(bbbp_chemberta2_features_test.iloc[:, 2:])
bbbp_chemberta2_features_t.reset_index(drop=True, inplace=True)

bbbp_molformer_features_t  = pd.DataFrame(bbbp_molformer_features_test.iloc[:, 1:])
bbbp_molformer_features_t.reset_index(drop=True, inplace=True)

bbbp_molbert_features_t = pd.DataFrame(bbbp_molbert_features_test.iloc[:, 1:])
bbbp_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([bbbp_chemberta2_prob_test, bbbp_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([bbbp_molformer_prob_test, bbbp_molformer_features_t], axis=1)
molbert_X_test = pd.concat([bbbp_molbert_prob_test, bbbp_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict using the best models from valid2
chemberta_pred_test = best_model_chemberta.predict(chemberta_X_test_scaled)
molformer_pred_test = best_model_molformer.predict(molformer_X_test_scaled)
molbert_pred_test = best_model_molbert.predict(molbert_X_test_scaled)

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='bce_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='bce_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='bce_molbert')

# Now concatenate the series with the test set probabilities
bbbp_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # BCE for Chemberta
    molformer_pred_test_series,                     # BCE for Molformer
    molbert_pred_test_series,                       # BCE for Molbert
    bbbp_chemberta2_prob_test['chemberta2'],        # Chemberta test probabilities
    bbbp_molformer_prob_test['molformer'],          # Molformer test probabilities
    bbbp_molbert_prob_test['molbert']               # Molbert test probabilities
], axis=1)

bbbp_X_ensemble_test.columns = ['bce_chemberta', 'bce_molformer', 'bce_molbert', 'chemberta2', 'molformer', 'molbert']

# optional for evaluation
bbbp_y_ensemble_test = bbbp_chemberta2_test['p_np']

In [10]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bbbp_X_ensemble_valid2_scaled = scaler.fit_transform(bbbp_X_ensemble_valid2)
bbbp_X_ensemble_test_scaled = scaler.transform(bbbp_X_ensemble_test)

# transform back to dataframe
bbbp_X_ensemble_valid2_scaled = pd.DataFrame(bbbp_X_ensemble_valid2_scaled, columns=bbbp_X_ensemble_valid2.columns)
bbbp_X_ensemble_test_scaled = pd.DataFrame(bbbp_X_ensemble_test_scaled, columns=bbbp_X_ensemble_test.columns)


In [11]:
# # save the scaled data to csv
# bbbp_X_ensemble_valid2_scaled.to_csv('./processed_data/bbbp_X_ensemble_valid2_scaled_aux.csv', index=False)
# bbbp_X_ensemble_test_scaled.to_csv('./processed_data/bbbp_X_ensemble_test_scaled_aux.csv', index=False)

# # save the target values to csv
# bbbp_y_ensemble_valid2.to_csv('./processed_data/bbbp_y_ensemble_valid2.csv', index=False)
# bbbp_y_ensemble_test.to_csv('./processed_data/bbbp_y_ensemble_test.csv', index=False)

In [12]:
# # use lasso regression to train the ensemble model
# from sklearn.linear_model import LogisticRegressionCV

# # cross validation for strength of regularization
# lasso_cv = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', max_iter=5000, random_state=0, scoring='roc_auc')

# # Fit the model
# lasso_cv.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# # Predict the test set
# bbbp_lasso_pred = lasso_cv.predict(bbbp_X_ensemble_test_scaled)
# bbbp_lasso_probs = lasso_cv.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

# # Calculate the metrics
# bbbp_lasso_metrics = {
#     "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_lasso_pred),
#     "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_lasso_pred),
#     "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_lasso_probs),
#     "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_lasso_probs)
# }

# bbbp_lasso_metrics

In [13]:
# coefs = pd.Series(lasso_cv.coef_[0], index=bbbp_X_ensemble_valid2.columns)

# # Filter to get the selected features
# selected_features = coefs[coefs != 0].index.tolist()

# # Check if 'chemberta2', 'molformer', 'molbert' are in the selected features, if not, add them
# for model in ['chemberta2', 'molformer', 'molbert']:
#     if model not in selected_features:
#         selected_features.append(model)

# print("Selected Features:", selected_features)
# # check how many features are selected
# print(len(selected_features))

In [14]:
from group_lasso import LogisticGroupLasso
import numpy as np

# Create an array that specifies the group for each feature
# Assuming the number of features in your dataset:
n_features = bbbp_X_ensemble_valid2_scaled.shape[1]
groups = np.zeros(n_features, dtype=int)
groups[:3] = 1  # First three features as one group
groups[3:] = 2  # Rest of the features as another group

# Initialize the Logistic Group Lasso model
group_lasso = LogisticGroupLasso(
    groups=groups,
    group_reg=0.05,  # Regularization strength for the groups
    l1_reg=0,        # No L1 regularization
    scale_reg='none', # Do not automatically scale the regularization
    supress_warning=True,
    tol=1e-2,
    random_state=0
)

# Fit the model
group_lasso.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_group_lasso_pred = group_lasso.predict(bbbp_X_ensemble_test_scaled)
bbbp_group_lasso_probs = group_lasso.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

bbbp_two_groups_lasso_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_group_lasso_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_group_lasso_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_group_lasso_probs),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_group_lasso_probs)
}

bbbp_two_groups_lasso_metrics

{'Accuracy': 0.9166666666666666,
 'F1 Score': 0.9492537313432836,
 'ROC-AUC': 0.9621116043119848,
 'PR-AUC': 0.991338592573501}

In [16]:
from group_lasso import LogisticGroupLasso
import numpy as np

# Create an array that specifies the group for each feature
# Assuming the number of features in your dataset:
n_features = bbbp_X_ensemble_valid2_scaled.shape[1]
groups = np.zeros(n_features, dtype=int)

# Setting the groups according to the new specification:
groups[:3] = 1      # First three features as one group
groups[3:3+384] = 2 # Next 384 features as second group
groups[3+384:3+384+768] = 3 # Next 768 features as third group
groups[3+384+768:] = 4 # Remaining 768 features as fourth group

# Initialize the Logistic Group Lasso model
group_lasso = LogisticGroupLasso(
    groups=groups,
    group_reg=0.05,  # Regularization strength for the groups
    l1_reg=0,        # No L1 regularization
    scale_reg='none', # Do not automatically scale the regularization
    supress_warning=True,
    tol=1e-2,
    random_state=0
)

# Fit the model
group_lasso.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_group_lasso_pred = group_lasso.predict(bbbp_X_ensemble_test_scaled)
bbbp_group_lasso_probs = group_lasso.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

bbbp_four_groups_lasso_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_group_lasso_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_group_lasso_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_group_lasso_probs),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_group_lasso_probs)
}

bbbp_four_groups_lasso_metrics

{'Accuracy': 0.8725490196078431,
 'F1 Score': 0.9171974522292994,
 'ROC-AUC': 0.9224793912492074,
 'PR-AUC': 0.9720416623742437}

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Define the model with elasticnet penalty
elastic_net_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=5000, random_state=0)

# Use fewer discrete values for alpha and l1_ratio
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range
l1_ratios = [0.1, 0.5, 0.9]  # Reduced to three points, emphasizing edges and midpoint

# Convert alphas to Cs for the parameter grid (since C is the inverse of alpha)
Cs = [1/alpha for alpha in alphas]

# Create a more concise grid search using 5-fold cross-validation
params = {
    'C': Cs,
    'l1_ratio': l1_ratios
}

grid_search = GridSearchCV(elastic_net_model, param_grid=params, cv=5, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# Best model after grid search
bbbp_best_elastic_model = grid_search.best_estimator_
print(grid_search.best_params_)

# Predict the test set
bbbp_elastic_pred = bbbp_best_elastic_model.predict(bbbp_X_ensemble_test_scaled)
bbbp_elastic_probs = bbbp_best_elastic_model.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
bbbp_elastic_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_elastic_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_elastic_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_elastic_probs),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_elastic_probs)
}

bbbp_elastic_metrics

{'C': 1.0, 'l1_ratio': 0.9}


{'Accuracy': 0.8676470588235294,
 'F1 Score': 0.918429003021148,
 'ROC-AUC': 0.9178820545339252,
 'PR-AUC': 0.9801500016939654}

In [19]:
# Access the coefficients from elastic net
coefs = pd.Series(bbbp_best_elastic_model.coef_[0], index=bbbp_X_ensemble_valid2.columns)

# Filter to get the selected features
selected_features = coefs[coefs != 0].index.tolist()

# Check if 'chemberta2', 'molformer', 'molbert' are in the selected features, if not, add them
for model in ['chemberta2', 'molformer', 'molbert']:
    if model not in selected_features:
        selected_features.append(model)

print("Selected Features:", selected_features)
print(len(selected_features))

Selected Features: ['chemberta2', 'molformer', 'molbert', 'chemberta2_feature_34', 'chemberta2_feature_57', 'chemberta2_feature_67', 'chemberta2_feature_71', 'chemberta2_feature_72', 'chemberta2_feature_85', 'chemberta2_feature_88', 'chemberta2_feature_89', 'chemberta2_feature_106', 'chemberta2_feature_135', 'chemberta2_feature_169', 'chemberta2_feature_173', 'chemberta2_feature_178', 'chemberta2_feature_183', 'chemberta2_feature_207', 'chemberta2_feature_292', 'chemberta2_feature_308', 'chemberta2_feature_320', 'chemberta2_feature_332', 'chemberta2_feature_357', 'chemberta2_feature_364', 'molformer_feature_21', 'molformer_feature_22', 'molformer_feature_32', 'molformer_feature_46', 'molformer_feature_73', 'molformer_feature_84', 'molformer_feature_113', 'molformer_feature_119', 'molformer_feature_126', 'molformer_feature_138', 'molformer_feature_141', 'molformer_feature_174', 'molformer_feature_177', 'molformer_feature_184', 'molformer_feature_196', 'molformer_feature_205', 'molformer

In [20]:
bbbp_X_ensemble_valid2_selected = bbbp_X_ensemble_valid2_scaled
bbbp_X_ensemble_test_selected = bbbp_X_ensemble_test_scaled

# check shapes
print(bbbp_X_ensemble_valid2_selected.shape)
print(bbbp_X_ensemble_test_selected.shape)

(401, 1923)
(204, 1923)


In [21]:
# Initialize and train the SVM model
from sklearn.svm import SVC
import random

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)

bbbp_svm_model = SVC(probability=True, random_state=0)
bbbp_svm_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_svm_pred = bbbp_svm_model.predict(bbbp_X_ensemble_test_selected)
bbbp_svm_probs = bbbp_svm_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_svm_metrics = {
    'Accuracy': accuracy_score(bbbp_y_ensemble_test, bbbp_svm_pred),
    'F1 Score': f1_score(bbbp_y_ensemble_test, bbbp_svm_pred),
    'ROC-AUC': roc_auc_score(bbbp_y_ensemble_test, bbbp_svm_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_svm_probs[:, 1])
}

bbbp_svm_metrics

{'Accuracy': 0.9019607843137255,
 'F1 Score': 0.9401197604790419,
 'ROC-AUC': 0.9326252377932784,
 'PR-AUC': 0.9832504131011419}

In [22]:
# initailize and use a 5-fold cross-validation to tune the hyperparameters of a random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

bbbp_rf_model = RandomForestClassifier(random_state=0)

bbbp_rf_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_rf_best_pred = bbbp_rf_model.predict(bbbp_X_ensemble_test_selected)
bbbp_rf_best_probs = bbbp_rf_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_rf_best_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_rf_best_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_rf_best_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_rf_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_rf_best_probs[:, 1])
}

bbbp_rf_best_metrics

{'Accuracy': 0.9019607843137255,
 'F1 Score': 0.9408284023668639,
 'ROC-AUC': 0.8998890298034242,
 'PR-AUC': 0.9628275188980567}

In [23]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)

# Define the hyperparameter space using continuous distributions
bbbp_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    model = xgb.XGBClassifier(**params, random_state=0)
    
    # Cross-validated AUC score as the objective
    roc_auc = make_scorer(roc_auc_score, response_method=None)
    score = cross_val_score(model, bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2, scoring=roc_auc, cv=5)
    
    # Minimize the negative ROC AUC score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
bbbp_xgb_best_params = fmin(fn=objective, 
                          space=bbbp_xgb_hyperopt_space, 
                          algo=tpe.suggest, 
                          max_evals=50, 
                          trials=trials,
                          rstate=np.random.default_rng(0),  # Seed for hyperopt
                          early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bbbp_xgb_best_params)

 26%|██▌       | 13/50 [00:41<01:57,  3.18s/trial, best loss: -1.0]              
Best hyperparameters: {'colsample_bytree': 0.6685136034824202, 'learning_rate': 0.11277987288069052, 'max_depth': 4.0, 'n_estimators': 150.0, 'subsample': 0.7210771337355122}


In [24]:
# Convert parameters obtained from Hyperopt to the correct data type
bbbp_xgb_best_params['n_estimators'] = int(bbbp_xgb_best_params['n_estimators'])
bbbp_xgb_best_params['max_depth'] = int(bbbp_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
bbbp_xgb_model = xgb.XGBClassifier(**bbbp_xgb_best_params, random_state=0)
bbbp_xgb_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_xgb_best_pred = bbbp_xgb_model.predict(bbbp_X_ensemble_test_selected)
bbbp_xgb_best_probs = bbbp_xgb_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_xgb_best_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_xgb_best_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_xgb_best_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_xgb_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_xgb_best_probs[:, 1])
}

bbbp_xgb_best_metrics

{'Accuracy': 0.8137254901960784,
 'F1 Score': 0.8972972972972973,
 'ROC-AUC': 0.11770767279644898,
 'PR-AUC': 0.7578679830885408}

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params):
    kf = KFold(n_splits=5)
    roc_aucs = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            roc_auc = roc_auc_score(y_val_tensor.numpy(), outputs.numpy())
            roc_aucs.append(roc_auc)

    avg_roc_auc = np.mean(roc_aucs)
    return {'loss': -avg_roc_auc, 'status': STATUS_OK}  # Maximize ROC AUC by minimizing the negative ROC AUC

# Hyperparameter space
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

X = bbbp_X_ensemble_valid2_selected
y = bbbp_y_ensemble_valid2

# Run Bayesian optimization
trials = Trials()
bbbp_nn_best_params = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            rstate=np.random.default_rng(0),  # Seed for hyperopt
            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bbbp_nn_best_params)


 20%|██        | 10/50 [01:07<04:29,  6.74s/trial, best loss: -0.9755714008654952]
Best hyperparameters: {'dropout_rate': 0.4175299871186762, 'learning_rate': 0.00024043206714908388, 'num_layers': 2.0, 'num_neurons': 180.0}


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import roc_auc_score

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Convert parameters to the correct format if necessary
bbbp_nn_best_params = {
    'num_layers': int(bbbp_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons': int(bbbp_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': bbbp_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': bbbp_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(bbbp_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(bbbp_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(bbbp_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(bbbp_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=bbbp_X_ensemble_valid2_selected.shape[1], num_layers=bbbp_nn_best_params['num_layers'], 
                 num_neurons=bbbp_nn_best_params['num_neurons'], dropout_rate=bbbp_nn_best_params['dropout_rate'])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=bbbp_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs > 0.5).float()

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predictions.numpy())
    roc_auc = roc_auc_score(y_test_tensor.numpy(), outputs.numpy())
    pr_auc = average_precision_score(y_test_tensor.numpy(), outputs.numpy())

    bbbp_nn_metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

bbbp_nn_metrics

{'Accuracy': 0.8921568627450981,
 'F1 Score': 0.9325153374233128,
 'ROC-AUC': 0.9169308814204185,
 'PR-AUC': 0.970362406202526}

In [27]:
# report all the metrics for bbbp
bbbp_metrics_results["LASSO"] = bbbp_lasso_metrics
bbbp_metrics_results["Group Lasso (Two Groups)"] = bbbp_two_groups_lasso_metrics
bbbp_metrics_results["Group Lasso (Four Groups)"] = bbbp_four_groups_lasso_metrics
bbbp_metrics_results["Elastic Net"] = bbbp_elastic_metrics
bbbp_metrics_results["SVM"] = bbbp_svm_metrics
bbbp_metrics_results["Random Forest"] = bbbp_rf_best_metrics
bbbp_metrics_results["XGBoost"] = bbbp_xgb_best_metrics
bbbp_metrics_results["Neural Network"] = bbbp_nn_metrics

bbbp_metrics_df = pd.DataFrame(bbbp_metrics_results).T

# keep 3 digits after the decimal point
bbbp_metrics_df = bbbp_metrics_df.round(3)

# export as csv
bbbp_metrics_df.to_csv('./split3_bbbp_metrics_aux.csv')